In [1]:
import torch.nn as nn

import sys
sys.path.append('../modules')

from vae_designer import VAEDesigner
from custom_vae import VAEDesign, get_vae_and_opt

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## phase 0: prepare data
using `vae_fast_train`

## phase 1: design a vae and stores its design as json
using `VAEDesigner` and `VAEDesign.save_as_json`

Step 1. Design encoder and decoder using VAEDesigner

In [3]:
enc_des = VAEDesigner(input_shape=(17, 16, 16), num_layers=2, up_sample=False)

In [9]:
dec_designer = VAEDesigner(input_shape=(128, 2, 2), num_layers=2, up_sample=True)

Step 2. Add any other information to encoder design / decoder design.m

In [20]:
decoder_design = dec_designer.design
decoder_design.update({'final_activation':'sigmoid'})

Step 3. Create design and save as json. This step ensures that you can check on that design later without using the vae-designer widget.

In [14]:
vae_design = VAEDesign(enc_des.design, decoder_design, h_dim=512, z_dim=64, unflatten_out_shape=(128, 2, 2))
vae_design.save_as_json('designs/pcgml_vae_v1_design.json')

## phase 2: create vae and its optimizer
using `get_vae_and_opt` from the design json

Step 4. Create a new trainable vae using the saved design json (don't have to create the VAEDesign object next time) and device.

In [17]:
vae, opt = get_vae_and_opt(design_json_fpath='designs/pcgml_vae_v1_design.json', dev='cpu')

In [18]:
vae, opt

(VAE(
   (encoder): Sequential(
     (block0-conv2d): Conv2d(17, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
     (block0-bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (block0-lrelu): LeakyReLU(negative_slope=0.2)
     (block1-conv2d): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), bias=False)
     (block1-bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (flatten): Flatten()
   )
   (fc1): Linear(in_features=512, out_features=64, bias=True)
   (fc2): Linear(in_features=512, out_features=64, bias=True)
   (fc3): Linear(in_features=64, out_features=512, bias=True)
   (decoder): Sequential(
     (unflatten): UnFlatten()
     (block0-convtranpose2d): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), output_padding=(1, 1), bias=False)
     (block0-bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (block0-relu): ReLU()
     (block1-convtranpose2d)

## phase 3: train vae and visualize training progress
using `vae_fast_train` (callbacks, tensorboard)

In [25]:
import torch

In [30]:
torch.utils

<module 'torch.utils' from '/Users/yangzhihan/anaconda/envs/deeplearning_googlecloud/lib/python3.6/site-packages/torch/utils/__init__.py'>

In [6]:
import numpy as np

writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)
    writer.add_image('Generations/1', torch.from_numpy(np.random.random((3, 5, 5))))